In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr').getOrCreate()

In [3]:
data = spark.read.csv('Ecommerce_Customers.csv',inferSchema=True,header=True)

In [4]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [6]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [7]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [8]:
# Spark needs data in two or one column only 

In [9]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [10]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [12]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership'],
                            outputCol='features')

In [13]:
output = assembler.transform(data)

In [14]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [16]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [17]:
final_data = output.select('features','Yearly Amount Spent')

In [18]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [19]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                340|
|   mean| 496.24268420891974|
| stddev|  80.14217499407525|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [20]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                160|
|   mean| 505.84066561414147|
| stddev|   77.3697012086838|
|    min|  275.9184206503857|
|    max|  744.2218671047146|
+-------+-------------------+



In [22]:
# Create Model
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [23]:
lrModel = lr.fit(train_data)

In [25]:
# coefficient and intercept
print('Coefficient: {} Intercept: {}'.format(lrModel.coefficients,lrModel.intercept))

Coefficient: [25.769918106230296,38.45541112290801,0.3424484264068093,61.6817205202894] Intercept: -1047.0258615939508


In [26]:
test_results = lrModel.evaluate(test_data)

In [27]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-11.133336356601035|
| 11.233069423440497|
|  -5.70650593938791|
|-12.617312793886583|
| 11.246160360581598|
|0.08078225060478417|
|  22.52391678251388|
| -4.640111264161533|
| -3.876321314746349|
| -8.353215090998162|
|-1.4701532175398597|
| -6.676411540743061|
|  2.267132598158298|
| -8.678738753556615|
| 12.189591609384763|
|-13.369050038737896|
|-17.314131997176673|
| -8.326535073070772|
| 5.8649067228164995|
| 16.891971979159393|
+-------------------+
only showing top 20 rows



In [28]:
unlabeled_data = test_data.select('features')

In [29]:
predictions = lrModel.transform(unlabeled_data)

In [30]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...|331.06220615979464|
|[30.7377203726281...| 450.5476727727894|
|[31.0613251567161...| 493.2619639972895|
|[31.0662181616375...|461.55060600156094|
|[31.1695067987115...| 416.1103704417112|
|[31.2606468698795...| 421.2458490063466|
|[31.2834474760581...| 569.2571726431536|
|[31.5147378578019...|494.45259926062295|
|[31.5171218025062...|279.79474196513206|
|[31.5261978982398...|  417.447741283336|
|[31.5761319713222...| 542.6967372068682|
|[31.7207699002873...|  545.451345018766|
|[31.7366356860502...| 494.6663136573736|
|[31.8279790554652...|448.68148630049814|
|[31.9262720263601...|380.01534183494164|
|[31.9365486184489...| 440.5684349340661|
|[31.9563005605233...| 564.4400637443755|
|[32.0085045178551...|451.52375610182617|
|[32.0305497162129...| 588.4095766957953|
|[32.0609143984100...| 610.7113467338556|
+--------------------+------------

In [32]:
print('RMSE: {}'.format(test_results.rootMeanSquaredError))
print('MSE: {}'.format(test_results.meanSquaredError))

RMSE: 9.953122639009855
MSE: 99.0646502671705
